In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

::::{margin}
:::{card} Phase space factor with rotated cut
TR-980
^^^
We rotate the cut in the definition of the phase space factor $\rho(s)$, but **do not rotate the path** of the dispersion integral.
+++
🚧&nbsp;[compwa.github.io#204](https://github.com/ComPWA/compwa.github.io/pull/204)
:::
::::

# Phase space factor with rotated cut

In [ ]:
%pip install -q numpy==1.24.4 scipy==1.10.1

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import quad_vec as integrate

warnings.filterwarnings("ignore")

In [ ]:
ϵ = 1e-8


def rho_cm(s, m1, m2, phi: float = 0) -> np.ndarray:
    return -1j * dispersion_integral(s, m1, m2, phi)


def dispersion_integral(s, m1, m2, phi: float = 0) -> np.ndarray:
    s_thr = (m1 + m2) ** 2
    func = lambda x: rho(x, m1, m2, phi) / ((x - ϵ * 1j - s_thr) * (x - ϵ * 1j - s))
    return (s - s_thr) / np.pi * integrate(func, s_thr, np.inf)[0]


def rho(s, m1, m2, phi: float = 0) -> np.ndarray:
    return sqrt((s - (m1 + m2) ** 2) * (s - (m1 - m2) ** 2), phi) / s


def sqrt(z, phi: float = 0) -> np.ndarray:
    return np.exp(-0.5j * phi) * np.sqrt(z * np.exp(phi * 1j))

In [ ]:
%config InlineBackend.figure_formats = ['svg']
resolution = 100
x = np.linspace(0, 3, num=resolution)
m1 = 0.1
m2 = 0.7
phi = -np.pi / 6

y1 = rho(x, m1, m2, phi)
y2 = rho_cm(x, m1, m2, phi)
assert not any(np.isnan(y2)), "Contains NaN values!"

fig, ax = plt.subplots()
ax.axhline(0, c="black", lw=0.4)
ax.axvline(0, c="black", lw=0.4)
ax.plot(x, y1.real, label=R"$\rho$", lw=4, alpha=0.5)
ax.plot(x, y2.real, label=R"$\rho^\mathrm{CM}$")
ax.axvline((m1 - m2) ** 2, c="red", ls="dotted", label="$(m_1-m_2)^2$")
ax.axvline((m1 + m2) ** 2, c="black", ls="dotted", label="$(m_1+m_2)^2$")
ax.set_ylim(-0.3, +1.6)
ax.set_xlabel("$s$")
ax.set_title(Rf"$\phi={phi / np.pi:.4g}\pi$")
ax.legend()
fig.tight_layout()
plt.show()

In [ ]:
%config InlineBackend.figure_formats = ['png']
y = np.linspace(-1, +1, num=resolution)
X, Y = np.meshgrid(x, y)
Z = X + Y * 1j

T = rho(Z, m1, m2, phi)
fig, ax = plt.subplots()
fig.suptitle("Phase space factor with rotated cut")
ax.set_xlabel(R"$\mathrm{Re}\,s$")
ax.set_ylabel(R"$\mathrm{Im}\,s$")
ax.set_title(Rf"$\phi={phi / np.pi:.4g}\pi$")
mesh = ax.pcolormesh(X, Y, T.real, cmap=plt.cm.coolwarm, vmin=-5, vmax=5)
plt.colorbar(mesh, ax=ax, pad=0.02)
fig.tight_layout()
plt.show()

In [ ]:
%config InlineBackend.figure_formats = ['png']
T = rho_cm(Z, m1, m2, phi)
fig, ax = plt.subplots()
fig.suptitle("Dispersion integral over rotated phase space factor")
ax.set_xlabel(R"$\mathrm{Re}\,s$")
ax.set_ylabel(R"$\mathrm{Im}\,s$")
ax.set_title(Rf"$\phi={phi / np.pi:.4g}\pi$")
mesh = ax.pcolormesh(X, Y, T.real, cmap=plt.cm.coolwarm)
plt.colorbar(mesh, ax=ax, pad=0.02)
fig.tight_layout()
plt.show()

:::{error}
Rotating the cut in the square root definition for $\rho$ still does not rotate the cut in the dispersion integral! 😭
:::